# Options B/C: Deploy with Existing Resources

Use this notebook when you have existing Azure resources and want to:

- **Option B**: Deploy function CODE only to an existing Function App
- **Option C**: Deploy a NEW Function App with existing backend resources

## Cross-Resource-Group Support

All existing resource parameters support resources in different resource groups:
- Storage accounts in shared data resource groups
- Cosmos DB managed by database teams
- Key Vault in security resource groups
- Log Analytics in monitoring resource groups (even different subscriptions!)

## Step 1: Configure Variables

Update with your existing resource information:

In [ ]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

$SUBSCRIPTION_ID = "363ef5d1-0e77-4594-a530-f51af23dbf8c"
$LOCATION = "westus2"

# Target resource group for Function App
$RESOURCE_GROUP = "rg-docprocessing-functions-dev"

# Existing Storage Account
$EXISTING_STORAGE_ACCOUNT = "aimldatastore"
$EXISTING_STORAGE_RG = "rg-dlz-aiml-stack-dev"

# Existing Document Intelligence
$EXISTING_DOC_INTEL = "docservendpointdev"
$EXISTING_DOC_INTEL_RG = "rg-dlz-aiml-stack-dev"

# Existing Cosmos DB
$EXISTING_COSMOS = "cosmosdb-dlz-east2-sandbox"
$EXISTING_COSMOS_RG = "rg-dlz-cosmosdb-east2-sandbox"

# Existing Key Vault
$EXISTING_KEY_VAULT = "aiml-stack-keyvault-dev"
$EXISTING_KEY_VAULT_RG = "rg-dlz-aiml-stack-dev"

# Existing Synapse (optional - leave empty if not using)
$EXISTING_SYNAPSE = "synapse-sandbox-east2-dlz"  # or ""
$EXISTING_SYNAPSE_RG = "rg-sandbox-demo-east2"   # or ""

# Existing Log Analytics (optional - for centralized logging)
$EXISTING_LOG_ANALYTICS = "alz-dev-dataObservability-logAnalyticsWorkspace"
$EXISTING_LOG_ANALYTICS_RG = "rg-alz-dev-logging"
$EXISTING_LOG_ANALYTICS_SUBSCRIPTION = "a60a2fdd-c133-4845-9beb-31f470bf3ef5"  # Only if in different subscription

# ============================================
# DEPLOYMENT MODE
# ============================================
# Option B: Use existing Function App (set name below, $DEPLOY_NEW_FUNC = $false)
# Option C: Deploy new Function App (leave name empty, $DEPLOY_NEW_FUNC = $true)

$EXISTING_FUNC_APP = "docproc-func-dev"  # Leave empty to deploy new Function App (Option C)
$DEPLOY_NEW_FUNC = $false  # Set to $false for Option B

# ============================================

az account set --subscription $SUBSCRIPTION_ID

Write-Host "Configuration:" -ForegroundColor Green
Write-Host "  Mode: $(if ($DEPLOY_NEW_FUNC) { 'Option C - Deploy new Function App' } else { 'Option B - Use existing Function App' })"
Write-Host "  Target RG: $RESOURCE_GROUP"

## Step 2: Verify Existing Resources

Check that all existing resources can be accessed.

In [ ]:
Write-Host "Verifying existing resources..." -ForegroundColor Cyan

# Check Storage Account
Write-Host "`nStorage Account: $EXISTING_STORAGE_ACCOUNT" -ForegroundColor Yellow
$storage = az storage account show --name $EXISTING_STORAGE_ACCOUNT --resource-group $EXISTING_STORAGE_RG --output json 2>&1
if ($LASTEXITCODE -eq 0) {
    Write-Host "  [OK] Storage account found" -ForegroundColor Green
} else {
    Write-Host "  [ERROR] Storage account not found" -ForegroundColor Red
}

# Check Document Intelligence
Write-Host "`nDocument Intelligence: $EXISTING_DOC_INTEL" -ForegroundColor Yellow
$docintel = az cognitiveservices account show --name $EXISTING_DOC_INTEL --resource-group $EXISTING_DOC_INTEL_RG --output json 2>&1
if ($LASTEXITCODE -eq 0) {
    Write-Host "  [OK] Document Intelligence found" -ForegroundColor Green
} else {
    Write-Host "  [ERROR] Document Intelligence not found" -ForegroundColor Red
}

# Check Cosmos DB
Write-Host "`nCosmos DB: $EXISTING_COSMOS" -ForegroundColor Yellow
$cosmos = az cosmosdb show --name $EXISTING_COSMOS --resource-group $EXISTING_COSMOS_RG --output json 2>&1
if ($LASTEXITCODE -eq 0) {
    Write-Host "  [OK] Cosmos DB found" -ForegroundColor Green
} else {
    Write-Host "  [ERROR] Cosmos DB not found" -ForegroundColor Red
}

# Check Key Vault
Write-Host "`nKey Vault: $EXISTING_KEY_VAULT" -ForegroundColor Yellow
$kv = az keyvault show --name $EXISTING_KEY_VAULT --resource-group $EXISTING_KEY_VAULT_RG --output json 2>&1
if ($LASTEXITCODE -eq 0) {
    Write-Host "  [OK] Key Vault found" -ForegroundColor Green
} else {
    Write-Host "  [ERROR] Key Vault not found" -ForegroundColor Red
}

# Check Synapse Workspace (if specified)
if ($EXISTING_SYNAPSE) {
    Write-Host "`nSynapse Workspace: $EXISTING_SYNAPSE" -ForegroundColor Yellow
    $synapse = az synapse workspace show --name $EXISTING_SYNAPSE --resource-group $EXISTING_SYNAPSE_RG --output json 2>&1
    if ($LASTEXITCODE -eq 0) {
        $synapseObj = $synapse | ConvertFrom-Json
        Write-Host "  [OK] Synapse workspace found" -ForegroundColor Green
        Write-Host "  Location: $($synapseObj.location)" -ForegroundColor Gray
        Write-Host "  SQL Endpoint: $($synapseObj.connectivityEndpoints.sql)" -ForegroundColor Gray
        Write-Host "  Dev Endpoint: $($synapseObj.connectivityEndpoints.dev)" -ForegroundColor Gray
        
        # List Spark pools
        $sparkPools = az synapse spark pool list --workspace-name $EXISTING_SYNAPSE --resource-group $EXISTING_SYNAPSE_RG --output json 2>&1
        if ($LASTEXITCODE -eq 0) {
            $pools = $sparkPools | ConvertFrom-Json
            Write-Host "  Spark Pools: $($pools.Count) found" -ForegroundColor Gray
            foreach ($pool in $pools) {
                Write-Host "    - $($pool.name) ($($pool.nodeSize), Spark $($pool.sparkVersion))" -ForegroundColor Gray
            }
        }
    } else {
        Write-Host "  [ERROR] Synapse workspace not found" -ForegroundColor Red
    }
} else {
    Write-Host "`nSynapse Workspace: Not configured (optional)" -ForegroundColor Gray
}

# Check Log Analytics (if specified)
if ($EXISTING_LOG_ANALYTICS) {
    Write-Host "`nLog Analytics: $EXISTING_LOG_ANALYTICS" -ForegroundColor Yellow
    $logSub = if ($EXISTING_LOG_ANALYTICS_SUBSCRIPTION) { $EXISTING_LOG_ANALYTICS_SUBSCRIPTION } else { $SUBSCRIPTION_ID }
    $la = az monitor log-analytics workspace show --workspace-name $EXISTING_LOG_ANALYTICS --resource-group $EXISTING_LOG_ANALYTICS_RG --subscription $logSub --output json 2>&1
    if ($LASTEXITCODE -eq 0) {
        Write-Host "  [OK] Log Analytics workspace found" -ForegroundColor Green
    } else {
        Write-Host "  [ERROR] Log Analytics workspace not found" -ForegroundColor Red
    }
} else {
    Write-Host "`nLog Analytics: Not configured (optional)" -ForegroundColor Gray
}

## Step 3a: Option B - Deploy Code to Existing Function App

Use this section if you have an existing Function App.

In [ ]:
# ============================================
# OPTION B: Deploy to Existing Function App
# ============================================

if (-not $DEPLOY_NEW_FUNC -and $EXISTING_FUNC_APP) {
    Write-Host "Configuring existing Function App: $EXISTING_FUNC_APP" -ForegroundColor Cyan
    
    # Get Document Intelligence endpoint and key
    $DOC_INTEL_ENDPOINT = az cognitiveservices account show `
        --name $EXISTING_DOC_INTEL `
        --resource-group $EXISTING_DOC_INTEL_RG `
        --query properties.endpoint -o tsv
    
    $DOC_INTEL_KEY = az cognitiveservices account keys list `
        --name $EXISTING_DOC_INTEL `
        --resource-group $EXISTING_DOC_INTEL_RG `
        --query key1 -o tsv
    
    # Get Cosmos DB endpoint
    $COSMOS_ENDPOINT = az cosmosdb show `
        --name $EXISTING_COSMOS `
        --resource-group $EXISTING_COSMOS_RG `
        --query documentEndpoint -o tsv
    
    # Configure app settings
    Write-Host "Setting Function App configuration..." -ForegroundColor Yellow
    
    az functionapp config appsettings set `
        --name $EXISTING_FUNC_APP `
        --resource-group $RESOURCE_GROUP `
        --settings `
            DOC_INTEL_ENDPOINT=$DOC_INTEL_ENDPOINT `
            DOC_INTEL_API_KEY=$DOC_INTEL_KEY `
            COSMOS_ENDPOINT=$COSMOS_ENDPOINT `
            COSMOS_DATABASE="DocumentsDB" `
            COSMOS_CONTAINER="ExtractedDocuments" `
            KEY_VAULT_NAME=$EXISTING_KEY_VAULT `
            DEFAULT_MODEL_ID="prebuilt-layout" `
            MAX_CONCURRENT_REQUESTS="10" `
        --output none
    
    Write-Host "Configuration complete" -ForegroundColor Green
} else {
    Write-Host "Skipping Option B - Use Option C cells below" -ForegroundColor Yellow
}

In [ ]:
# Deploy function code (Option B)
if (-not $DEPLOY_NEW_FUNC -and $EXISTING_FUNC_APP) {
    Write-Host "Deploying function code to: $EXISTING_FUNC_APP" -ForegroundColor Cyan
    
    Set-Location -Path "E:\Repos\GitHub\MyDemoRepos\FormExtraction\src\functions"
    func azure functionapp publish $EXISTING_FUNC_APP --python
    Set-Location -Path "E:\Repos\GitHub\MyDemoRepos\FormExtraction"
    
    Write-Host "Function code deployed" -ForegroundColor Green
}

## Step 3b: Option C - Deploy New Function App with Existing Backend

Use this section to deploy a new Function App that connects to your existing resources.

In [ ]:
# ============================================
# OPTION C: Deploy New Function App
# ============================================

if ($DEPLOY_NEW_FUNC) {
    Write-Host "Deploying new Function App with existing backend..." -ForegroundColor Cyan
    
    # Check if using configured parameter file
    $CONFIGURED_PARAM = "infra/parameters/configured/existing.bicepparam"
    $DEFAULT_PARAM = "infra/parameters/existing.bicepparam"
    
    Set-Location -Path "$PSScriptRoot/.."
    
    if (Test-Path $CONFIGURED_PARAM) {
        $PARAM_FILE = $CONFIGURED_PARAM
        Write-Host "Using configured parameter file: $PARAM_FILE" -ForegroundColor Green
    } else {
        $PARAM_FILE = $DEFAULT_PARAM
        Write-Host "Using default parameter file: $PARAM_FILE" -ForegroundColor Yellow
        Write-Host "TIP: Copy to infra/parameters/configured/ for your custom values" -ForegroundColor Yellow
    }
    
    # Preview deployment
    Write-Host "`nPreviewing deployment (what-if)..." -ForegroundColor Cyan
    
    az deployment sub what-if `
        --subscription $SUBSCRIPTION_ID `
        --location $LOCATION `
        --template-file infra/main.bicep `
        --parameters $PARAM_FILE
} else {
    Write-Host "Skipping Option C - Using Option B above" -ForegroundColor Yellow
}

In [ ]:
# Deploy infrastructure (Option C)
if ($DEPLOY_NEW_FUNC) {
    Write-Host "Deploying infrastructure..." -ForegroundColor Cyan
    
    az deployment sub create `
        --subscription $SUBSCRIPTION_ID `
        --location $LOCATION `
        --template-file infra/main.bicep `
        --parameters $PARAM_FILE
    
    if ($LASTEXITCODE -eq 0) {
        Write-Host "Infrastructure deployed successfully!" -ForegroundColor Green
        
        # Get new Function App name
        $FUNC_APP_NAME = az deployment sub show `
            --subscription $SUBSCRIPTION_ID `
            --name main `
            --query properties.outputs.functionAppName.value -o tsv
        
        Write-Host "New Function App: $FUNC_APP_NAME" -ForegroundColor Green
    } else {
        Write-Host "Deployment failed" -ForegroundColor Red
    }
}

In [ ]:
# Deploy function code (Option C)
if ($DEPLOY_NEW_FUNC -and $FUNC_APP_NAME) {
    Write-Host "Deploying function code to: $FUNC_APP_NAME" -ForegroundColor Cyan
    
    Set-Location -Path "$PSScriptRoot/../src/functions"
    func azure functionapp publish $FUNC_APP_NAME --python
    Set-Location -Path "$PSScriptRoot/.."
    
    Write-Host "Function code deployed" -ForegroundColor Green
}

In [ ]:
write-host $PSScriptRoot

## Step 4: Set Up Synapse Integration

If using an existing Synapse workspace, deploy the pipeline artifacts.

In [ ]:
if ($EXISTING_SYNAPSE) {
    Write-Host "Deploying Synapse artifacts to: $EXISTING_SYNAPSE" -ForegroundColor Cyan
    
    # Get Function App name
    $FUNC_NAME = if ($EXISTING_FUNC_APP) { $EXISTING_FUNC_APP } else { $FUNC_APP_NAME }
    
    # Build URLs
    $STORAGE_URL = "https://$EXISTING_STORAGE_ACCOUNT.blob.core.windows.net"
    $FUNC_URL = "https://$FUNC_NAME.azurewebsites.net"
    $KV_URL = "https://$EXISTING_KEY_VAULT.vault.azure.net/"
    
    Write-Host "  Storage URL: $STORAGE_URL"
    Write-Host "  Function URL: $FUNC_URL"
    Write-Host "  Key Vault URL: $KV_URL"
    
    Set-Location -Path "E:\Repos\GitHub\MyDemoRepos\FormExtraction\"
    
    .\scripts\Deploy-SynapseArtifacts.ps1 `
        -WorkspaceName $EXISTING_SYNAPSE `
        -ResourceGroup $EXISTING_SYNAPSE_RG `
        -StorageAccountUrl $STORAGE_URL `
        -FunctionAppUrl $FUNC_URL `
        -KeyVaultUrl $KV_URL `
        -DeploymentMode external-github `
        -ExternalRepoUrl "https://github.com/fgarofalo56/csa-synapse.git" `
        -ExternalRepoRootFolder "/" `
        -GitHubBranch "dev-collaboration" 

} else {
    Write-Host "No existing Synapse workspace specified - skipping" -ForegroundColor Yellow
}

## Step 5: Configure Authentication

Store the Function App key in Key Vault and grant Synapse access.

In [ ]:
# Get the Function App name to use
$FUNC_NAME = if ($EXISTING_FUNC_APP) { $EXISTING_FUNC_APP } else { $FUNC_APP_NAME }
$FUNC_RG = $RESOURCE_GROUP

Write-Host "Configuring authentication for: $FUNC_NAME" -ForegroundColor Cyan

# Get Function App host key
$FUNCTION_KEY = az functionapp keys list `
    --name $FUNC_NAME `
    --resource-group $FUNC_RG `
    --query "functionKeys.default" -o tsv

if ($FUNCTION_KEY) {
    # Store in Key Vault
    Write-Host "Storing function key in Key Vault..." -ForegroundColor Yellow
    
    az keyvault secret set `
        --vault-name $EXISTING_KEY_VAULT `
        --name "FunctionAppHostKey" `
        --value $FUNCTION_KEY `
        --output none
    
    Write-Host "Function key stored successfully" -ForegroundColor Green
} else {
    Write-Host "Failed to get function key" -ForegroundColor Red
}

In [ ]:
# Grant Synapse access to Key Vault
if ($EXISTING_SYNAPSE) {
    Write-Host "Granting Synapse access to Key Vault..." -ForegroundColor Cyan
    
    $SYNAPSE_IDENTITY = az synapse workspace show `
        --name $EXISTING_SYNAPSE `
        --resource-group $EXISTING_SYNAPSE_RG `
        --query identity.principalId -o tsv
    
    Write-Host "Synapse Identity: $SYNAPSE_IDENTITY"
    
    az keyvault set-policy `
        --name $EXISTING_KEY_VAULT `
        --object-id $SYNAPSE_IDENTITY `
        --secret-permissions get list `
        --output none
    
    Write-Host "Key Vault access granted" -ForegroundColor Green
}

## Step 6: Verify Deployment

Test the Function App health endpoint.

In [ ]:
$FUNC_NAME = if ($EXISTING_FUNC_APP) { $EXISTING_FUNC_APP } else { $FUNC_APP_NAME }
$FUNC_URL = "https://$FUNC_NAME.azurewebsites.net"
$FUNCTION_KEY = az functionapp keys list `
    --name $FUNC_NAME `
    --resource-group $RESOURCE_GROUP `
    --query "functionKeys.default" -o tsv

try {
    $response = Invoke-RestMethod -Uri "$FUNC_URL/api/health?code=$FUNCTION_KEY" -Method Get
    Write-Host "Health check passed!" -ForegroundColor Green
    $response | ConvertTo-Json
} catch {
    Write-Host "Health check failed: $_" -ForegroundColor Red
}

## Deployment Complete!

Your pipeline is now connected to existing resources. Next steps:

1. **Test the pipeline** - See `05-Synapse-Pipeline.ipynb`
2. **Set up local development** - See `03-Local-Development.ipynb`
3. **Test and publish in Synapse Studio** - Open Synapse Studio → Manage → Linked services → Test connections → Publish all